In [1]:
# Published dashboard: https://your-deployed-dashboard-url (Password: yourpassword)
# CP 321 – Data Visualization Assignment 7

import re
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [83]:
url = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals"
tables = pd.read_html(url)

finals_df = None
for table in tables:
    cols = [str(col) for col in table.columns]
    if "Years won" in cols and "Team" in cols and "Years runners-up" in cols:
        finals_df = table
        break

country_iso = {
    "Uruguay": "URY",
    "Argentina": "ARG",
    "Italy": "ITA",
    "Czechoslovakia": "TCH",
    "Hungary": "HUN",
    "Brazil": "BRA",
    "Sweden": "SWE",
    "Germany": "DEU",
    "England": "GBR",
    "Netherlands": "NLD",
    "France": "FRA",
    "Spain": "ESP",
    "Croatia": "HRV"
}
finals_df["Team"] = finals_df["Team"].map(country_iso)

print(finals_df)

   Team  Winners  Runners-up  Total finals                     Years won  \
0   BRA        5           2             7  1958, 1962, 1970, 1994, 2002   
1   DEU        4           4             8        1954, 1974, 1990, 2014   
2   ITA        4           2             6        1934, 1938, 1982, 2006   
3   ARG        3           3             6              1978, 1986, 2022   
4   FRA        2           2             4                    1998, 2018   
5   URY        2           0             2                    1930, 1950   
6   GBR        1           0             1                          1966   
7   ESP        1           0             1                          2010   
8   NLD        0           3             3                             —   
9   HUN        0           2             2                             —   
10  TCH        0           2             2                             —   
11  SWE        0           1             1                             —   
12  HRV     

In [89]:
wins_count_df["Team"] = finals_df["Team"]
wins_count_df["Wins"] = finals_df["Winners"]
wins_count_df["Runner ups"] = finals_df["Runners-up"]

print(wins_count_df)

   Team  Wins  Runner ups
0   BRA     5           2
1   DEU     4           4
2   ITA     4           2
3   ARG     3           3
4   FRA     2           2
5   URY     2           0
6   GBR     1           0
7   ESP     1           0
8   NLD     0           3
9   HUN     0           2
10  TCH     0           2
11  SWE     0           1
12  HRV     0           1


In [91]:
winners_list = []
for _, row in finals_df.iterrows():
    team_code = row["Team"]
    years_won_str = row["Years won"]
    if years_won_str and years_won_str != "-":
        for y in years_won_str.split(","):
            y = y.strip()
            if y.isdigit():
                winners_list.append({"Year": int(y), "Winner": team_code})
winners_df = pd.DataFrame(winners_list)

runners_list = []
for _, row in finals_df.iterrows():
    team_code = row["Team"]
    years_runners_str = row["Years runners-up"]
    if years_runners_str and years_runners_str != "-":
        for y in years_runners_str.split(","):
            y = y.strip()
            if y.isdigit():
                runners_list.append({"Year": int(y), "Runner-up": team_code})
runners_df = pd.DataFrame(runners_list)

finals_yearly_df = pd.merge(winners_df, runners_df, on="Year", how="outer")

print(finals_yearly_df)

    Year Winner Runner-up
0   1958    BRA       SWE
1   1962    BRA       TCH
2   1970    BRA       ITA
3   1994    BRA       ITA
4   2002    BRA       DEU
5   1954    DEU       HUN
6   1974    DEU       NLD
7   1990    DEU       ARG
8   2014    DEU       ARG
9   1934    ITA       TCH
10  1938    ITA       HUN
11  1982    ITA       DEU
12  2006    ITA       FRA
13  1978    ARG       NLD
14  1986    ARG       DEU
15  2022    ARG       FRA
16  1998    FRA       BRA
17  2018    FRA       HRV
18  1930    URY       ARG
19  1950    URY       BRA
20  1966    GBR       DEU
21  2010    ESP       NLD


In [115]:
app = dash.Dash(__name__)
app.title = "FIFA World Cup Dashboard"

fig_choropleth = px.choropleth(
    wins_count_df,
    locations="Team",         
    color="Wins",
    hover_name="Team",
    hover_data=["Runner ups"],
    color_continuous_scale=px.colors.sequential.Plasma,
    title="FIFA World Cup Wins by Country"
)

app.layout = html.Div([
    html.H1("FIFA World Cup Winners and Runner-ups Dashboard"),

    dcc.Graph(
        id="worldcup-choropleth",
        figure=fig_choropleth
    ),

    html.Div([
        html.Label("Select a country to view number of wins:"),
        dcc.Dropdown(
            id="country-dropdown",
            options=[{"label": code, "value": code} for code in wins_count_df["Team"].unique()],
            placeholder="Select a country"
        ),
        html.Div(id="country-wins-output", style={"marginTop": 20, "fontSize": "20px"})
    ], style={"width": "48%", "display": "inline-block", "verticalAlign": "top"}),

    html.Div([
        html.Label("Select a year to view final match result:"),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": str(y), "value": y} for y in sorted(finals_yearly_df["Year"].unique())],
            placeholder="Select a year"
        ),
        html.Div(id="year-result-output", style={"marginTop": 20, "fontSize": "20px"})
    ], style={"width": "48%", "display": "block", "verticalAlign": "top"})
])

@app.callback(
    Output("country-wins-output", "children"),
    Input("country-dropdown", "value")
)
def update_country_wins(selected_country):
    if selected_country:
        row = wins_count_df[wins_count_df["Team"] == selected_country]
        if not row.empty:
            wins = row["Wins"].values[0]
            runner_ups = row["Runner ups"].values[0]
            return f"{selected_country} has won the World Cup {wins} time(s) and been runner-up {runner_ups} time(s)."
        return f"No record found for {selected_country}."
    return "Select a country to see its number of wins and runner-ups."

@app.callback(
    Output("year-result-output", "children"),
    Input("year-dropdown", "value")
)
def update_year_result(selected_year):
    if selected_year:
        row = finals_yearly_df[finals_yearly_df["Year"] == selected_year]
        if not row.empty:
            winner = row["Winner"].values[0] if "Winner" in row.columns else "Unknown"
            runner_up = row["Runner-up"].values[0] if "Runner-up" in row.columns else "Unknown"
            return f"In {selected_year}, {winner} won the World Cup, defeating {runner_up}."
        else:
            return f"No data for the year {selected_year}."
    return "Select a year to view the final match result."

if __name__ == "__main__":
    app.run(debug=True)